In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import MisrToolkit as mtk # needs python3.6
# from mpl_toolkits.basemap import Basemap # needs python 3.8?
import sys # to check python version
import os
import glob

# Extract data from MISR-estimated roughness files by using MTK library. Should use python3.6 for this purpose 

## Plot Transect 

In [2]:
atm_file = "3cams_insample_atm.csv"
atm_dir = "/home/ehsan/misr_lab/MISR-SeaIceRoughness/post_processing"
atm_data_fp = os.path.join(atm_dir, atm_file)

In [3]:
transect_small = pd.read_csv(atm_data_fp)
transect_small

,Unnamed: 0,lat,lon,Ca_r,Cf_r,An_r,rms
0,1,76.235487,230.712293,0.66574,1.03182,0.53599,15.24
1,2,76.235648,230.710186,0.66574,1.03182,0.53599,22.32
2,3,76.235474,230.712462,0.66574,1.03182,0.53599,17.44
3,4,76.235619,230.714797,0.66574,1.03182,0.53599,14.30
4,5,76.235781,230.712690,0.66574,1.03182,0.53599,14.95
...,...,...,...,...,...,...,...
9995,9996,75.999639,230.398900,0.58862,0.95719,0.41134,5.65
9996,9997,75.999793,230.401122,0.58862,0.95719,0.41134,5.24
9997,9998,75.999951,230.399059,0.58862,0.95719,0.41134,6.27
9998,9999,76.000109,230.396997,0.58163,0.95175,0.41600,5.40


In [4]:
transect_small = transect_small.iloc[0:100]
transect_small.iloc[0]

Unnamed: 0      1.000000
lat            76.235487
lon           230.712293
Ca_r            0.665740
Cf_r            1.031820
An_r            0.535990
rms            15.240000
Name: 0, dtype: float64

In [5]:
roughness_dir = "/media/ehsan/6T_part1/2016/april_2016/14528_apr2016/project_april_2016_3cam/roughness_predicted_from_PH"


In [ ]:
# extract MISR-estimated roughness for the same latLon locations and plot them together with ATM
transect_from_misr = []
for irow in range(transect_small.shape[0]): # each row==LatLon might fall in several blocks==POB, we open each block and will average all at the end
    
    print("DF row: %s" %irow)
    lat_lon = transect_small[['lat','lon']].iloc[irow]
    print(lat_lon)
    
    path_list = mtk.latlon_to_path_list(lat_lon[0], lat_lon[1])
    print(path_list)
    
    roughness_per_pixel_list = []
    for path in path_list:
        # Q- now that we have found many paths, how figure out to select which path?
        bls = mtk.latlon_to_bls(path, 275 , lat_lon[0], lat_lon[1])
        print(bls)
        
        # use P and B to find roughness file, open it, and extract value for LS, append it to a list, and maybe finally append the list to DF as the last column
        if path < 100:
            print(path)
            path_str = str(path).zfill(3)
            print(path_str)
            
        roughness_filepattern = "roughness_toa_refl_P"+path_str+"_*"+"_B0"+str(bls[0])+".dat"
        print(roughness_filepattern)
        rough_file_fp = os.path.join(roughness_dir, roughness_filepattern)
        print(rough_file_fp)
        
        # check if file is found on machine
        roughness_file_found_list = glob.glob(os.path.join(roughness_dir, roughness_filepattern))
        print("roughness file found: %d" %len(roughness_file_found_list))
        
        if (len(roughness_file_found_list) > 1):
            raise ValueError('more than 1 file found, there is a problem with this code and it has to be reviewed.')
        
        if (len(roughness_file_found_list) <= 0):
            print("roughness file not found- continue to next path")
            continue
            
        # open roughness file and read pixel value- chat binary format to 2D array and read pixel value
        rough_2d_arr = np.fromfile(roughness_file_found_list[0], dtype=np.double)[0:1048576].reshape((512,-1))
        print(rough_2d_arr.shape)
        
        pixel_row = bls[1]
        pixel_col = bls[2]
        roughness_value = rough_2d_arr[int(pixel_row), int(pixel_col)]
        if roughness_value > 0:
            roughness_per_pixel_list.append(roughness_value)
    
    print("========================== extracted rough pixels ===========================")
    print(roughness_per_pixel_list)
    mean_roughness_per_pixel = sum(roughness_per_pixel_list)/len(roughness_per_pixel_list)
    roughness_per_pixel_list.clear()
    print(roughness_per_pixel_list)
    print(mean_roughness_per_pixel)
    print("\n")
    transect_from_misr.append(mean_roughness_per_pixel)


DF row: 0
lat     76.235487
lon    230.712293
Name: 0, dtype: float64
[62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148]
(29, 264.625, 9.251968383789062)
62
062
roughness_toa_refl_P062_*_B029.dat
/media/ehsan/6T_part1/2016/april_2016/14528_apr2016/project_april_2016_3cam/roughness_predicted_from_PH/roughness_toa_refl_P062_*_B029.dat
roughness file found: 1
(512, 2048)
(29, 200.7333984375, 144.29833984375)
63
063
roughness_toa_refl_P063_*_B029.dat
/media/ehsan/6T_part1/2016/april_2016/14528_apr2016/project_april_2016_3cam/roughness_predicted_from_PH/roughness_toa_refl_P063_*_B029.dat
roughness file found: 1
(512, 2048)
(29, 133.357421875, 277.6050720214844)
64
064
roughness_toa_refl_P064_*_B029.dat
/media/ehsan/6T_part1/2016/april_2016/14528_apr2016/project_april_2016_3cam/roughness_predicted_from_PH/roughness_toa_refl_P064_*_B029.dat
roughness file found: 1
(512, 2048)
(2

roughness file found: 0
roughness file not found- continue to next path
========================== extracted rough pixels ===========================
[14.194068667590345, 15.373619840823261, 24.02334692091305, 23.59528380555814, 16.69361206047467, 15.128831458225752, 19.556698987392696, 14.912331505916127, 16.05009424488754, 21.673652625350886, 16.100130848513004]
[]
17.936515542331403


DF row: 1
lat     76.235648
lon    230.710186
Name: 1, dtype: float64
[62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148]
(29, 264.650390625, 9.039459228515625)
62
062
roughness_toa_refl_P062_*_B029.dat
/media/ehsan/6T_part1/2016/april_2016/14528_apr2016/project_april_2016_3cam/roughness_predicted_from_PH/roughness_toa_refl_P062_*_B029.dat
roughness file found: 1
(512, 2048)
(29, 200.7646484375, 144.08660888671875)
63
063
roughness_toa_refl_P063_*_B029.dat
/media/ehsan/6T_part1/2016/april_

roughness file found: 0
roughness file not found- continue to next path
========================== extracted rough pixels ===========================
[15.399651705813014, 15.373619840823261, 24.02334692091305, 23.59528380555814, 16.60303187499345, 14.509459886068912, 19.556698987392696, 14.912331505916127, 17.005038411085632, 21.673652625350886, 16.185548047580877]
[]
18.07615123740873


DF row: 2
lat     76.235474
lon    230.712462
Name: 2, dtype: float64
[62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148]
(29, 264.623046875, 9.269027709960938)
62
062
roughness_toa_refl_P062_*_B029.dat
/media/ehsan/6T_part1/2016/april_2016/14528_apr2016/project_april_2016_3cam/roughness_predicted_from_PH/roughness_toa_refl_P062_*_B029.dat
roughness file found: 1
(512, 2048)
(29, 200.7314453125, 144.31533813476562)
63
063
roughness_toa_refl_P063_*_B029.dat
/media/ehsan/6T_part1/2016/april_

roughness file found: 1
(512, 2048)
(29, 200.5791015625, 144.49227905273438)
63
063
roughness_toa_refl_P063_*_B029.dat
/media/ehsan/6T_part1/2016/april_2016/14528_apr2016/project_april_2016_3cam/roughness_predicted_from_PH/roughness_toa_refl_P063_*_B029.dat
roughness file found: 1
(512, 2048)
(29, 133.197265625, 277.79486083984375)
64
064
roughness_toa_refl_P064_*_B029.dat
/media/ehsan/6T_part1/2016/april_2016/14528_apr2016/project_april_2016_3cam/roughness_predicted_from_PH/roughness_toa_refl_P064_*_B029.dat
roughness file found: 1
(512, 2048)
(29, 62.3740234375, 409.271484375)
65
065
roughness_toa_refl_P065_*_B029.dat
/media/ehsan/6T_part1/2016/april_2016/14528_apr2016/project_april_2016_3cam/roughness_predicted_from_PH/roughness_toa_refl_P065_*_B029.dat
roughness file found: 1
(512, 2048)
(28, 500.1533203125, 538.8365478515625)
66
066
roughness_toa_refl_P066_*_B028.dat
/media/ehsan/6T_part1/2016/april_2016/14528_apr2016/project_april_2016_3cam/roughness_predicted_from_PH/roughness_t

062
roughness_toa_refl_P062_*_B029.dat
/media/ehsan/6T_part1/2016/april_2016/14528_apr2016/project_april_2016_3cam/roughness_predicted_from_PH/roughness_toa_refl_P062_*_B029.dat
roughness file found: 1
(512, 2048)
(29, 200.609375, 144.28036499023438)
63
063
roughness_toa_refl_P063_*_B029.dat
/media/ehsan/6T_part1/2016/april_2016/14528_apr2016/project_april_2016_3cam/roughness_predicted_from_PH/roughness_toa_refl_P063_*_B029.dat
roughness file found: 1
(512, 2048)
(29, 133.2333984375, 277.5838623046875)
64
064
roughness_toa_refl_P064_*_B029.dat
/media/ehsan/6T_part1/2016/april_2016/14528_apr2016/project_april_2016_3cam/roughness_predicted_from_PH/roughness_toa_refl_P064_*_B029.dat
roughness file found: 1
(512, 2048)
(29, 62.4150390625, 409.0615234375)
65
065
roughness_toa_refl_P065_*_B029.dat
/media/ehsan/6T_part1/2016/april_2016/14528_apr2016/project_april_2016_3cam/roughness_predicted_from_PH/roughness_toa_refl_P065_*_B029.dat
roughness file found: 1
(512, 2048)
(28, 500.2001953125, 5

...

(512, 2048)
(29, 200.361328125, 144.24447631835938)
63
063
roughness_toa_refl_P063_*_B029.dat
/media/ehsan/6T_part1/2016/april_2016/14528_apr2016/project_april_2016_3cam/roughness_predicted_from_PH/roughness_toa_refl_P063_*_B029.dat
roughness file found: 1
(512, 2048)
(29, 132.986328125, 277.54150390625)
64
064
roughness_toa_refl_P064_*_B029.dat
/media/ehsan/6T_part1/2016/april_2016/14528_apr2016/project_april_2016_3cam/roughness_predicted_from_PH/roughness_toa_refl_P064_*_B029.dat
roughness file found: 1
(512, 2048)
(29, 62.1689453125, 409.01275634765625)
65
065
roughness_toa_refl_P065_*_B029.dat
/media/ehsan/6T_part1/2016/april_2016/14528_apr2016/project_april_2016_3cam/roughness_predicted_from_PH/roughness_toa_refl_P065_*_B029.dat
roughness file found: 1
(512, 2048)
(28, 499.955078125, 538.5726318359375)
66
066
roughness_toa_refl_P066_*_B028.dat
/media/ehsan/6T_part1/2016/april_2016/14528_apr2016/project_april_2016_3cam/roughness_predicted_from_PH/roughness_toa_refl_P066_*_B028.dat


(512, 2048)
(29, 200.392578125, 144.03265380859375)
63
063
roughness_toa_refl_P063_*_B029.dat
/media/ehsan/6T_part1/2016/april_2016/14528_apr2016/project_april_2016_3cam/roughness_predicted_from_PH/roughness_toa_refl_P063_*_B029.dat
roughness file found: 1
(512, 2048)
(29, 133.0224609375, 277.3305969238281)
64
064
roughness_toa_refl_P064_*_B029.dat
/media/ehsan/6T_part1/2016/april_2016/14528_apr2016/project_april_2016_3cam/roughness_predicted_from_PH/roughness_toa_refl_P064_*_B029.dat
roughness file found: 1
(512, 2048)
(29, 62.2109375, 408.80291748046875)
65
065
roughness_toa_refl_P065_*_B029.dat
/media/ehsan/6T_part1/2016/april_2016/14528_apr2016/project_april_2016_3cam/roughness_predicted_from_PH/roughness_toa_refl_P065_*_B029.dat
roughness file found: 1
(512, 2048)
(28, 500.0029296875, 538.3639526367188)
66
066
roughness_toa_refl_P066_*_B028.dat
/media/ehsan/6T_part1/2016/april_2016/14528_apr2016/project_april_2016_3cam/roughness_predicted_from_PH/roughness_toa_refl_P066_*_B028.dat

(512, 2048)
(29, 200.359375, 144.26190185546875)
63
063
roughness_toa_refl_P063_*_B029.dat
/media/ehsan/6T_part1/2016/april_2016/14528_apr2016/project_april_2016_3cam/roughness_predicted_from_PH/roughness_toa_refl_P063_*_B029.dat
roughness file found: 1
(512, 2048)
(29, 132.9833984375, 277.5588684082031)
64
064
roughness_toa_refl_P064_*_B029.dat
/media/ehsan/6T_part1/2016/april_2016/14528_apr2016/project_april_2016_3cam/roughness_predicted_from_PH/roughness_toa_refl_P064_*_B029.dat
roughness file found: 1
(512, 2048)
(29, 62.166015625, 409.030029296875)
65
065
roughness_toa_refl_P065_*_B029.dat
/media/ehsan/6T_part1/2016/april_2016/14528_apr2016/project_april_2016_3cam/roughness_predicted_from_PH/roughness_toa_refl_P065_*_B029.dat
roughness file found: 1
(512, 2048)
(28, 499.9521484375, 538.5897827148438)
66
066
roughness_toa_refl_P066_*_B028.dat
/media/ehsan/6T_part1/2016/april_2016/14528_apr2016/project_april_2016_3cam/roughness_predicted_from_PH/roughness_toa_refl_P066_*_B028.dat
ro

(512, 2048)
(29, 200.2060546875, 144.43914794921875)
63
063
roughness_toa_refl_P063_*_B029.dat
/media/ehsan/6T_part1/2016/april_2016/14528_apr2016/project_april_2016_3cam/roughness_predicted_from_PH/roughness_toa_refl_P063_*_B029.dat
roughness file found: 1
(512, 2048)
(29, 132.826171875, 277.7320251464844)
64
064
roughness_toa_refl_P064_*_B029.dat
/media/ehsan/6T_part1/2016/april_2016/14528_apr2016/project_april_2016_3cam/roughness_predicted_from_PH/roughness_toa_refl_P064_*_B029.dat
roughness file found: 1
(512, 2048)
(29, 62.00390625, 409.19903564453125)
65
065
roughness_toa_refl_P065_*_B029.dat
/media/ehsan/6T_part1/2016/april_2016/14528_apr2016/project_april_2016_3cam/roughness_predicted_from_PH/roughness_toa_refl_P065_*_B029.dat
roughness file found: 1
(512, 2048)
(28, 499.7861328125, 538.7544555664062)
66
066
roughness_toa_refl_P066_*_B028.dat
/media/ehsan/6T_part1/2016/april_2016/14528_apr2016/project_april_2016_3cam/roughness_predicted_from_PH/roughness_toa_refl_P066_*_B028.da

(512, 2048)
(29, 200.2373046875, 144.22726440429688)
63
063
roughness_toa_refl_P063_*_B029.dat
/media/ehsan/6T_part1/2016/april_2016/14528_apr2016/project_april_2016_3cam/roughness_predicted_from_PH/roughness_toa_refl_P063_*_B029.dat
roughness file found: 1
(512, 2048)
(29, 132.8623046875, 277.52105712890625)
64
064
roughness_toa_refl_P064_*_B029.dat
/media/ehsan/6T_part1/2016/april_2016/14528_apr2016/project_april_2016_3cam/roughness_predicted_from_PH/roughness_toa_refl_P064_*_B029.dat
roughness file found: 1
(512, 2048)
(29, 62.0458984375, 408.98907470703125)
65
065
roughness_toa_refl_P065_*_B029.dat
/media/ehsan/6T_part1/2016/april_2016/14528_apr2016/project_april_2016_3cam/roughness_predicted_from_PH/roughness_toa_refl_P065_*_B029.dat
roughness file found: 1
(512, 2048)
(28, 499.8330078125, 538.5457763671875)
66
066
roughness_toa_refl_P066_*_B028.dat
/media/ehsan/6T_part1/2016/april_2016/14528_apr2016/project_april_2016_3cam/roughness_predicted_from_PH/roughness_toa_refl_P066_*_B02

In [ ]:
# plot transect
# transect_small["rms"].plot(x=transect_small.index)#, y=transect_small)
# plt.plot(transect_from_misr)

x = transect_small.index
plt.plot(x, transect_small["rms"])
plt.plot(x, transect_from_misr)
plt.legend(["ATM obs","MISR-estimated"])

In [ ]:
transect_small.index

In [ ]:
transect_from_misr

## statistics

In [ ]:
transect_small.describe()

## histogram

In [ ]:
transect_small["rms"].hist(bins=20, alpha=1)